# Gensim

Authors: **Tasnima Sadekova**

Dr. Ilseyar Alimova

**Gensim** is billed as a Natural Language Processing package that does ‘Topic Modeling for Humans’. But it is practically much more than that. It is a leading and a state-of-the-art package for processing texts, working with *word vector models* and for building *topic models*.

But the width and scope of facilities to build and evaluate topic models are unparalleled in gensim, plus many more convenient facilities for text processing.

Also, another significant advantage with gensim is: it lets you handle large text files without having to load the entire file in memory.

In [1]:
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

import pandas as pd
import nltk
import re
import gensim
import os
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Dictionary and Corpus

**Dictionary** is an object that maps each word to a unique id.

The dictionary object is typically used to create a ‘bag of words’ Corpus. It is this Dictionary and the bag-of-words (Corpus) that are used as inputs to topic modeling and other models that Gensim specializes in.

*gensim.utils.simple_preprocess* Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.

In [3]:
# from a list of sentences

documents = ["If you use a car frequently, the first step to cutting",
             "down your emissions may well be to simply", 
             "fully consider the", 
             "alternatives available to you."
             ]

# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in documents]

# Create dictionary
dictionary = corpora.Dictionary(texts)

print(dictionary)
print(dictionary.token2id)

Dictionary(23 unique tokens: ['If', 'a', 'car', 'cutting', 'first']...)
{'If': 0, 'a': 1, 'car': 2, 'cutting': 3, 'first': 4, 'frequently,': 5, 'step': 6, 'the': 7, 'to': 8, 'use': 9, 'you': 10, 'be': 11, 'down': 12, 'emissions': 13, 'may': 14, 'simply': 15, 'well': 16, 'your': 17, 'consider': 18, 'fully': 19, 'alternatives': 20, 'available': 21, 'you.': 22}


With simple_preprocess

In [4]:
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in documents)  # deaccentuate
print(dictionary)
print(dictionary.token2id)

Dictionary(21 unique tokens: ['car', 'cutting', 'first', 'frequently', 'if']...)
{'car': 0, 'cutting': 1, 'first': 2, 'frequently': 3, 'if': 4, 'step': 5, 'the': 6, 'to': 7, 'use': 8, 'you': 9, 'be': 10, 'down': 11, 'emissions': 12, 'may': 13, 'simply': 14, 'well': 15, 'your': 16, 'consider': 17, 'fully': 18, 'alternatives': 19, 'available': 20}


Train your own word2vec model:

In [5]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=c98b6de8159b75308dac3c283de63659fc2c8363d628d9568d0646be4d177967
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [6]:
import wget

wget.download('https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv')

'unlabeledTrainData.tsv'

In [7]:
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [8]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


We remove links, html markup and non-alphabetic characters from the data, and then bring everything to lowercase and tokenize. The output is an array of sentences, each of which is an array of words. The tokenizer from the `nltk` library is used here.

In [9]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [10]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
    review_text = BeautifulSoup(review, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    words = review_text.lower().split()
    if remove_stopwords: # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # разбиваем обзор на предложения
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # применяем предыдущую функцию к каждому предложению
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [11]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


In [12]:
print(len(sentences))
print(sentences[0])

528199
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


Saving clean dataset:

In [13]:
!mkdir corpus

In [14]:
with open('corpus/sample.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

In [15]:
#from document
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in open('corpus/sample.txt', encoding='utf-8'))

In [16]:
len(dictionary)

11916

**Corpus** object that contains the word id and its frequency in each document.

The dictionary object is typically used to create a ‘bag of words’ Corpus. It is this Dictionary and the bag-of-words (Corpus) that are used as inputs to topic modeling and other models that Gensim specializes in.

- Bag of words

In [17]:
my_docs = ["Who let the dogs out?",
           "Who? Who? Who? Who?"]

# Tokenize the docs
tokenized_list = [simple_preprocess(doc) for doc in my_docs]

# Create the Corpus
dictionary = corpora.Dictionary()

#allow_update=True - add new words to dictionary
bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in tokenized_list]
print(bow_corpus)

print("Dictionary: ", dictionary.token2id)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(4, 4)]]
Dictionary:  {'dogs': 0, 'let': 1, 'out': 2, 'the': 3, 'who': 4}


The (0, 1) in line 1 means, the word with id=0 appears once in the 1st document.
Likewise, the (4, 4) in the second list item means the word with id 4 appears 4 times in the second document. And so on.

- TfIdf

In [18]:
from gensim import models
import numpy as np

documents = ["This is the first line",
             "This is the second sentence",
             "This third document"]

# Create the Dictionary and Corpus
mydict = corpora.Dictionary([simple_preprocess(line) for line in documents])
corpus = [mydict.doc2bow(simple_preprocess(line)) for line in documents]

# Show the Word Weights in Corpus
for doc in corpus:
    print([[mydict[id], freq] for id, freq in doc])
print()

# Create the TF-IDF model
tfidf = models.TfidfModel(corpus, smartirs='ntc')

# Show the TF-IDF weights
for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

[['first', 1], ['is', 1], ['line', 1], ['the', 1], ['this', 1]]
[['is', 1], ['the', 1], ['this', 1], ['second', 1], ['sentence', 1]]
[['this', 1], ['document', 1], ['third', 1]]

[['first', 0.66], ['is', 0.24], ['line', 0.66], ['the', 0.24]]
[['is', 0.24], ['the', 0.24], ['second', 0.66], ['sentence', 0.66]]
[['document', 0.71], ['third', 0.71]]


Save and load

In [19]:
# Save the Dict and Corpus
dictionary.save('mydict.dict')  # save dict to disk
corpora.MmCorpus.serialize('bow_corpus.mm', bow_corpus)  # save corpus to disk

In [20]:
# Load them back
loaded_dict = corpora.Dictionary.load('mydict.dict')

corpus = corpora.MmCorpus('bow_corpus.mm')
for line in corpus:
    print(line)

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0)]
[(4, 4.0)]


## Datasets

Gensim provides an inbuilt API to download popular text datasets and word embedding models.

A comprehensive list of available datasets and models is maintained [here](https://raw.githubusercontent.com/RaRe-Technologies/gensim-data/master/list.json).

Using the API to download the dataset is as simple as calling the ```api.load()``` method with the right data or model name.

In [21]:
import gensim.downloader as api

Short description

In [22]:
api.info('text8')
api.info('glove-wiki-gigaword-50')

{'num_records': 400000,
 'file_size': 69182535,
 'base_dataset': 'Wikipedia 2014 + Gigaword 5 (6B tokens, uncased)',
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/glove-wiki-gigaword-50/__init__.py',
 'license': 'http://opendatacommons.org/licenses/pddl/',
 'parameters': {'dimension': 50},
 'description': 'Pre-trained vectors based on Wikipedia 2014 + Gigaword, 5.6B tokens, 400K vocab, uncased (https://nlp.stanford.edu/projects/glove/).',
 'preprocessing': 'Converted to w2v format with `python -m gensim.scripts.glove2word2vec -i <fname> -o glove-wiki-gigaword-50.txt`.',
 'read_more': ['https://nlp.stanford.edu/projects/glove/',
  'https://nlp.stanford.edu/pubs/glove.pdf'],
 'checksum': 'c289bc5d7f2f02c6dc9f2f9b67641813',
 'file_name': 'glove-wiki-gigaword-50.gz',
 'parts': 1}

Dataset

In [23]:
dataset = api.load("text8")
data = [d for d in dataset]
print(data[0])

[==================================================] 100.0% 31.6/31.6MB downloaded
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing', 'interpretations',

Pretrained model

In [24]:
w2v_model = api.load("glove-wiki-gigaword-50")
w2v_model.most_similar('cat')

[==================================================] 100.0% 66.0/66.0MB downloaded


[('dog', 0.9218005537986755),
 ('rabbit', 0.8487821221351624),
 ('monkey', 0.804108202457428),
 ('rat', 0.7891963124275208),
 ('cats', 0.7865270376205444),
 ('snake', 0.7798910737037659),
 ('dogs', 0.7795814871788025),
 ('pet', 0.7792249917984009),
 ('mouse', 0.773166835308075),
 ('bite', 0.7728800177574158)]

In [25]:
w2v_model.most_similar('separator')

[('droplet', 0.6904144883155823),
 ('kritik', 0.6748565435409546),
 ('separators', 0.6572151780128479),
 ('infix', 0.6531262993812561),
 ('sieve', 0.6519265174865723),
 ('bcd', 0.6516676545143127),
 ('refrigerant', 0.651211142539978),
 ('infinitesimal', 0.6492906808853149),
 ('sublimation', 0.6465725302696228),
 ('sheaf', 0.6429159641265869)]

## Word2Vec

A word embedding model is a model that can provide numerical vectors for a given word. Using the Gensim’s downloader API, you can download pre-built word embedding models like word2vec, fasttext, GloVe and ConceptNet. These are built on large corpuses of commonly occurring text data such as wikipedia, google news etc.

The training algorithms in the Gensim package were actually ported from the original Word2Vec implementation by Google and extended with additional functionality.

This module implements the word2vec family of algorithms, using *highly optimized* C routines, data streaming and Pythonic interfaces.

**Parameters:**

- ```sentences``` - (iterable of iterables, optional) – The sentences iterable can be simply a *list of lists of tokens*, but for larger corpora, consider an *iterable* that streams the sentences directly from disk/network.
- ```corpus_file``` (str, optional) – Path to a corpus file in LineSentence format. You may use this argument instead of sentences to get performance boost. *Only one of sentences or corpus_file arguments need to be passed* 
- ```size``` = 100 - Dimensionality of the word vectors.
- ```window``` = 5 - Maximum distance between the current and predicted word within a sentence.
- ```min_count``` = 5 (int, optional) – Ignores all words with total frequency lower than this.
- ```workers``` = 3 (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines).
- ```sg``` = 0 ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
- ```hs``` = 0 ({0, 1}, optional) – If 1, [hierarchical softmax](https://paperswithcode.com/method/hierarchical-softmax) will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
- ```negative``` = 5 (int, optional) – If > 0, [negative sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/) will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
- ```max_vocab_size``` = None (int, optional) Limits the RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones.
- ```iter``` (int, optional) – Number of iterations (epochs) over the corpus.

In [26]:
import logging 
# Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [27]:
dataset = api.load("text8")
data = [d for d in dataset]

# Train Word2Vec model
model = Word2Vec(data)

```Word2Vec``` without ```sentences``` or ```corpus``` is initialization only, should be trained

In [28]:
model = Word2Vec()
model.build_vocab(data)
model.train(data, total_examples=model.corpus_count, epochs=5)

(62532507, 85026035)

**Save and load model**

In [29]:
model.save('w2v_newmodel')
model = Word2Vec.load('w2v_newmodel')

You can **continue training**

In [30]:
model.train([["hello", "world"]], total_examples=1, epochs=1)  # the warning is due to small corpus size

(2, 2)

**Word2vec input**


1) Parameter ```sentences```

Gensim’s word2vec expects a sequence of sentences as its input. Each sentence a list of words

   1.1 *List of list of tokens*

In [31]:
input1 = [['first', 'sentence'], ['second', 'sentence']]
model1 = Word2Vec(input1, min_count=1)

It also could be in any other language, e.g. Russian:

In [32]:
input2 = [['первое', 'предложение'], ['второе', 'предложение']]
model2 = Word2Vec(input2, min_count=1)

1.2 Gensim only requires that the input must provide sentences sequentially, when iterated over. *No need to keep everything in RAM*

In [33]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
                
input2 = MySentences('corpus') # a memory-friendly iterator
model = gensim.models.Word2Vec(input2)

See [BrownCorpus](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.BrownCorpus), [Text8Corpus](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus) or [LineSentence](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) in word2vec module for such examples.

BrownCorpus and Text8Corpus were implemented special for BrownCorpus and Text8 datasets. Text8 corpus, for example, consists of one line of cleaned and joined wikipedia articles.

LineSentence iterate over a file that contains sentences: one line = one sentence. Words must be already preprocessed and separated by whitespace.

In [34]:
import gensim
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence

input3 = LineSentence(datapath('lee_background.cor'))
model = gensim.models.Word2Vec(input3)

2. Parameter ```corpus_file``` - path to a corpus file in LineSentence format

If corpus is in right format, parameter corpus_file may be passed instead of last cell

## Exploring the model

- Extract the trained word vectors from model.wv:

In [35]:
w2v_model.wv['topic']

array([-0.04278 ,  0.79271 , -0.70087 , -0.023487,  0.24581 ,  0.24413 ,
       -0.10713 , -0.51894 , -0.17387 , -0.15821 , -0.90392 , -0.33753 ,
       -0.13262 ,  0.67051 ,  0.93457 ,  0.046388, -0.26368 , -0.23655 ,
        0.80884 ,  0.1048  ,  0.43985 , -0.068909,  0.83773 ,  1.0383  ,
        1.0378  , -1.0609  , -0.015392,  0.39162 , -0.1175  ,  0.40644 ,
        1.9836  , -0.41525 , -0.050877, -1.2321  , -0.69079 , -0.35601 ,
       -0.75549 ,  0.94668 , -0.84225 ,  0.095134, -0.092177, -0.13833 ,
       -0.30647 ,  0.89785 ,  0.071583,  0.31083 ,  0.88585 ,  1.1397  ,
       -0.19138 ,  0.27392 ], dtype=float32)

- Similarity

In [36]:
w2v_model.wv.most_similar(positive = ['topic'])

[('topics', 0.8801344037055969),
 ('discussion', 0.8731067180633545),
 ('debates', 0.7824684381484985),
 ('discussing', 0.7815945744514465),
 ('questions', 0.7759715914726257),
 ('debate', 0.7666820287704468),
 ('context', 0.7591570615768433),
 ('subject', 0.7455185055732727),
 ('question', 0.7334221601486206),
 ('focusing', 0.7311854958534241)]

In [37]:
model = w2v_model

In [38]:
print('Similarity between (walk, walking): ', model.wv.similarity('walk', 'walking'))
print('Similarity between (duck, ducks): ', model.wv.similarity('duck', 'ducks'))
print('Similarity between (banana, pear): ', model.wv.similarity('banana', 'pear'))
print()
print('Similarity between (banana, sky): ', model.wv.similarity('banana', 'sky'))
print('Similarity between (walk, lie): ', model.wv.similarity('walk', 'lie'))
print('Similarity between (dark, slow): ', model.wv.similarity('dark', 'slow'))

Similarity between (walk, walking):  0.9026177
Similarity between (duck, ducks):  0.6012516
Similarity between (banana, pear):  0.64588785

Similarity between (banana, sky):  0.081887454
Similarity between (walk, lie):  0.5613728
Similarity between (dark, slow):  0.3819838


In [39]:
model.wv.similarity('lol', 'cringe')

0.2712873

- Analogy

In [47]:
model.wv.most_similar(positive = ['lower', 'taller'], negative = ['low'])

[('upright', 0.5988052487373352),
 ('comfortably', 0.5728108882904053),
 ('sloped', 0.5719560384750366),
 ('erect', 0.5651558637619019),
 ('tallest', 0.5627383589744568),
 ('conical', 0.5587610602378845),
 ('pyramidal', 0.5548258423805237),
 ('unrounded', 0.5535889267921448),
 ('vertically', 0.5521244406700134),
 ('spire', 0.5520278215408325)]

In [41]:
model.wv.most_similar(positive = ['madrid', 'russia'], negative = ['spain'])

[('moscow', 0.8720105886459351),
 ('tehran', 0.7456655502319336),
 ('putin', 0.7295186519622803),
 ('kiev', 0.7200772166252136),
 ('belgrade', 0.6790421605110168),
 ('medvedev', 0.670237123966217),
 ('russian', 0.6692561507225037),
 ('kremlin', 0.6686785817146301),
 ('prague', 0.6660962700843811),
 ('iran', 0.6617439389228821)]

- Matching

In [42]:
print(model.wv.doesnt_match(['car', 'airplane', 'bed']), " doesn't match to [car, airplane]")
print(model.wv.doesnt_match(['red', 'blue', 'roof']), " doesn't match to [red, blue]")

bed  doesn't match to [car, airplane]
roof  doesn't match to [red, blue]


## Compare with other pretrained embeddings

In [57]:
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
word2vec_model300 = api.load('word2vec-google-news-300')

In [ ]:
glove_model300 = api.load('glove-wiki-gigaword-300')

To define which one performs better using the respective model's evaluate_word_analogies() 

Compute performance of the model on an analogy test set. The accuracy is reported (printed to log and returned as a score) for each section separately, plus there’s one aggregate summary at the end. 

Input:
- ```analogies``` (str) – Path to file, where lines are 4-tuples of words, split into sections by “: SECTION NAME” lines. See [this file](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/datasets/questions-words.txt) as example.



```
: capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
Athens Greece Beijing China
Athens Greece Berlin Germany
Athens Greece Bern Switzerland
Athens Greece Cairo Egypt
Athens Greece Canberra Australia
Athens Greece Hanoi Vietnam
Athens Greece Havana Cuba
Athens Greece Helsinki Finland
Athens Greece Islamabad Pakistan
```



Output:
- ```score``` (float) – The overall evaluation score on the entire evaluation set

- ```sections``` (list of dict of {str : str or list of tuple of (str, str, str, str)}) – Results broken down by each section of the evaluation set. Each dict contains the name of the section under the key ‘section’, and lists of correctly and incorrectly predicted 4-tuples of words under the keys ‘correct’ and ‘incorrect’.

In [60]:
wget.download("https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt")

'questions-words.txt'

In [61]:
f = open('questions-words.txt', 'r')
for i in range(5):
    print(f.readline())
f.close()

: capital-common-countries

Athens Greece Baghdad Iraq

Athens Greece Bangkok Thailand

Athens Greece Beijing China

Athens Greece Berlin Germany



In [ ]:
# w2v_accuracy
# print(word2vec_model300.evaluate_word_analogies(analogies='questions-words.txt')[0])

# fasttext_accuracy
print(fasttext_model300.evaluate_word_analogies(analogies='questions-words.txt')[0])

# GloVe accuracy
# print(glove_model300.evaluate_word_analogies(analogies='questions-words.txt')[0])

## Doc2Vec

Unlike Word2Vec, a [Doc2Vec](https://arxiv.org/pdf/1405.4053v2.pdf) model provides a vectorised representation of a group of words taken collectively as a single unit. It is not a simple average of the word vectors of the words in the sentence.

The training data for ```Doc2Vec``` should be a list of ```TaggedDocuments```. To create one, we pass a list of words and a unique integer as input to the ```models.doc2vec.TaggedDocument()```.

In [48]:
#prepare dataset
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield TaggedDocument(list_of_words, [i])

dataset = api.load("text8")
data = [d for d in dataset]

train_data = list(create_tagged_document(data))
print(train_data[:1])

[TaggedDocument(words=['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing', 'interpretations', 'of', 'what', 'this', 'means', 'anarchism', 'also', 'refers'

In [50]:
#Train model
model = Doc2Vec(vector_size=52, min_count=1, epochs=40)
model.build_vocab(train_data)
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [51]:
#Get document vector
print(model.infer_vector('gensim is really awesome'.split(' ')))

[-0.35838753  0.13483998  0.31623772 -0.04127119 -0.11664973 -0.16050605
 -0.26535895  0.00743161 -0.04268781 -0.12156912 -0.1636145  -0.2342846
 -0.22443374  0.04654546 -0.07742736  0.08364515 -0.18086684 -0.11550624
  0.07732766  0.19753113 -0.06688377 -0.17555381 -0.05061901  0.22579603
  0.07176768 -0.00981038  0.05803039 -0.23899661 -0.32887852  0.0858874
 -0.34418136 -0.04202545 -0.34178558  0.06547729 -0.48077238 -0.29418415
  0.07024238  0.1367531   0.16905828  0.10611473 -0.01455967  0.18052387
  0.11159025  0.07453012 -0.25353304 -0.07006625 -0.04323357  0.49873024
 -0.18346176  0.31183794  0.24306267  0.05934514]


## Word2Vec for Russian language

[RusVectores](https://rusvectores.org/ru/models/) provides a lot of different word embedding models in RusVectores.

In [52]:
import wget
import zipfile

# model -- ruscorpora_upos_cbow_300_20_2019
model_url = 'http://vectors.nlpl.eu/repository/20/180.zip'
wget.download(model_url)

'180.zip'

In [53]:
!unzip 180.zip

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [54]:
model_ru = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [55]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

In [56]:
for word in words: 
    if word in model_ru:
        print(word)
        print(model_ru[word][:10])
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            print(word, ': ', sim)
        print('\n')
    else:
        print('There is no word "%s" in the model!' % word)

день_NOUN
[ 1.805067   -0.877623   -1.0102742   2.8518744  -0.43311968 -3.7207692
 -3.4317713  -0.7634762  -4.9961104  -1.1313324 ]
неделя_NOUN :  0.7375995516777039
день_PROPN :  0.7067667245864868
месяц_NOUN :  0.7037326693534851
час_NOUN :  0.6643949747085571
утро_NOUN :  0.6526744961738586
вечер_NOUN :  0.6038411855697632
сутки_NOUN :  0.5923081040382385
воскресенье_NOUN :  0.5842781066894531
полдень_NOUN :  0.5743687152862549
суббота_NOUN :  0.5345946550369263


ночь_NOUN
[-0.10776415  0.32673436  0.52870405  2.1667976   0.7689093  -2.4214501
 -1.4222336  -2.972895    0.18769576 -0.05231643]
ночь_PROPN :  0.8310786485671997
вечер_NOUN :  0.7183678150177002
рассвет_NOUN :  0.696594774723053
ночи_NOUN :  0.6920218467712402
полночь_NOUN :  0.6704976558685303
ночь_VERB :  0.6615264415740967
утро_NOUN :  0.6263935565948486
ночной_ADJ :  0.6024709343910217
полдень_NOUN :  0.5835086107254028
сумерки_NOUN :  0.5671443343162537


человек_NOUN
[ 0.02881786 -0.7942778   2.4604542   2.2049303

Used materials: 
    
- https://www.machinelearningplus.com/nlp/gensim-tutorial/
- https://radimrehurek.com/gensim/models/word2vec.html
- https://radimrehurek.com/gensim/models/keyedvectors.html
- https://rare-technologies.com/word2vec-tutorial/